In [1]:
import os
import sys
import time

module_path = os.path.abspath(os.path.join('..'))
sys.path.append(module_path)

In [2]:
import pennylane as qml
from pennylane import numpy as np

from measurement_memory import *
from grouping import *

def read_Hamiltonian0(mol:str, grouping_type:str):
    path = os.path.abspath(os.path.join("{}_{}_Hamiltonian.txt".format(mol, grouping_type)))
    Hg = []
    Groups = []
    with open(path) as f:
        qubits, terms, groups = f.readline()[:-1].split(' ')
        
        group_ops = []
        group_coeffs = []
        T = 0
        G = 0
        for line in f.readlines():
            s = line.split(' ')
            if s[0] == '\n' or s[0] == None:
                Groups.append((group_ops, group_coeffs))
                Hg.append(qml.Hamiltonian(group_coeffs, str_to_Pauli(group_ops), grouping_type='commuting'))
                group_ops = []
                group_coeffs = []
                G += 1
            else:
                group_ops.append(s[0])
                if s[1][-1] == '\n': s[1] = s[1][:-1]
                group_coeffs.append(float(s[1]))
                T += 1
        assert T == int(terms) and G == int(groups), "Fail to read Hamiltonian correctly. Check if the last group end with '\n'."
    return Groups

In [3]:
def To_pennylane(Hg:list):
    obs = []
    coeffs = []
    for i in range(len(Hg)):
        for j,p in enumerate(str_to_Pauli(Hg[i][0])):
            obs.append(p)
            coeffs.append(Hg[i][1][j])
    qmlH = qml.Hamiltonian(coeffs, obs)
    return qmlH

## H2

In [4]:
# Define ansatz
N = 4
params = 2*np.pi*np.random.rand(N*2)
def ansatz(x):
    for i in range(N):
        qml.RX(x[i], wires=i)
    qml.CNOT(wires=[0,1])
    qml.CNOT(wires=[1,2])
    qml.CNOT(wires=[2,3])
    for i in range(N):
        qml.RX(x[i+N], wires=i)

#### 1. Original H

In [12]:
H = read_Hamiltonian0("H2", "GC")
qmlH = To_pennylane(H)

In [22]:
dev = qml.device("lightning.qubit", wires=N)
@qml.qnode(dev)
def circuit_raw(x):
    ansatz(x)
    return qml.expval(qmlH)

circuit_raw(params)

array(-0.08951336)

#### 2. QWC evaluation

In [14]:
H = read_Hamiltonian0("H2", "QWC")

In [11]:
dev = qml.device("lightning.qubit", wires=N, shots=999999)
@qml.qnode(dev)
def circuit_qwc(x, M):
    ansatz(x)
    single_qubit_basis_rotation(M)
    return qml.counts()

def evaluate(x):
    E = 0
    for h in H:
        M = is_QWC(h[0], return_basis=True)
        E += evaluate_eigenstate(circuit_qwc(params, M), h)
    return E

evaluate(params)

-0.08947988360151715

#### 3. GC evaluation

In [19]:
from Experiment_Molecular import measurement_rotation

H = read_Hamiltonian0("H2", "GC")
Hg, basis = basis_transformation(H)

In [21]:
dev = qml.device("lightning.qubit", wires=N, shots=999999)
@qml.qnode(dev)
def circuit_gc(x, T, Q):
    ansatz(x)
    measurement_rotation(T, Q)
    return qml.counts()

def evaluate(x):
    E = 0
    for i,h in enumerate(Hg):
        T, Q = basis[i]
        E += evaluate_eigenstate(circuit_gc(params, T, Q), h)
    return E

evaluate(params)

-0.0895212819630228

## H4

In [4]:
# Define ansatz
N = 8
params = 2*np.pi*np.random.rand(N*2)
def ansatz(x):
    for i in range(N):
        qml.RX(x[i], wires=i)
    qml.CNOT(wires=[0,1])
    qml.CNOT(wires=[1,2])
    qml.CNOT(wires=[2,3])
    for i in range(N):
        qml.RX(x[i+N], wires=i)

#### 1. Original H

In [5]:
H = read_Hamiltonian0("H4", "GC")
qmlH = To_pennylane(H)

In [6]:
dev = qml.device("lightning.qubit", wires=N)
@qml.qnode(dev)
def circuit_raw(x):
    ansatz(x)
    return qml.expval(qmlH)

circuit_raw(params)

array(0.9162883)

#### 2. QWC evaluation

In [7]:
H = read_Hamiltonian0("H4", "QWC")

In [9]:
dev = qml.device("lightning.qubit", wires=N, shots=999999)
@qml.qnode(dev)
def circuit_qwc(x, M):
    ansatz(x)
    single_qubit_basis_rotation(M)
    return qml.counts()

def evaluate(x):
    E = 0
    for h in H:
        M = is_QWC(h[0], return_basis=True)
        #print(M)
        E += evaluate_eigenstate(circuit_qwc(params, M), h)
    return E

evaluate(params)

0.9144130703743264

#### 3. GC evaluation

In [10]:
from Experiment_Molecular import measurement_rotation

H = read_Hamiltonian0("H4", "GC")
Hg, basis = basis_transformation(H)

In [11]:
dev = qml.device("lightning.qubit", wires=N, shots=999999)
@qml.qnode(dev)
def circuit_gc(x, T, Q):
    ansatz(x)
    measurement_rotation(T, Q)
    return qml.counts()

def evaluate(x):
    E = 0
    for i,h in enumerate(Hg):
        T, Q = basis[i]
        E += evaluate_eigenstate(circuit_gc(params, T, Q), h)
    return E

evaluate(params)

0.9145625886790075

In [13]:
H = read_Hamiltonian0("LiH", "GC")
print(len(H))

34


In [11]:
H = read_Hamiltonian0("LiH", "QWC")
print(len(H))

151
